In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

%matplotlib inline

In [2]:
df_train = pd.read_csv('../Data/cleaned_train.csv')
df_test = pd.read_csv('../Data/cleaned_test.csv')

In [4]:
df_train.drop(columns='Unnamed: 0', inplace=True)

In [5]:
df_test.drop(columns='Unnamed: 0', inplace=True)

In [6]:
df_train.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,Normal,13517,Pave,NaN,IR1,Lvl,...,0,0,No Pool,No Fence,None,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,No Pool,No Fence,None,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,No Pool,No Fence,None,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,No Pool,No Fence,None,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,No Pool,No Fence,None,0,3,2010,WD,138500
